# Stream Processing with Spark

## Assignment 7

### Assignment 7.1

In this assignment, you will run the [quick example from Apache Spark's structured streaming programming guide](http://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#quick-example). 
The first step involves opening a terminal and running the netcat command. You must start this command before running the Spark code. You can open a terminal in JupyterLab by selecting *File -> New -> Terminal* from the menu. Start the netcat programming running on port 9999 by using the following command. 

```
nc -lk 9999
```

With netcat still running in the terminal, come back to this notebook and run the following code. 

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession \
    .builder \
    .appName("Assignment 7.1") \
    .getOrCreate()

lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

# Split the lines into words
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

# Generate running word count
wordCounts = words.groupBy("word").count()

try:
    query = wordCounts \
        .writeStream \
        .outputMode("complete") \
        .format("console") \
        .start()

    query.awaitTermination()
except KeyboardInterrupt:
    print('Stopping query')

Now you can go back to the terminal and enter text values. You can input any text that you like. You can use [The Project Gutenberg EBook of The Odyssey, by Homer](https://www.gutenberg.org/files/1727/1727-0.txt) if you can't think of any sample text. After entering the text in the terminal, you should output within this notebook. 

### Assignment 7.2

Modify the word count query so that the streaming query only returns results where the word count is greater than two. Test your code by typing repeated words into the netcat program. 